# Setup

## Load Packages

In [4]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import openai
import re
import time
from scripts.prepare_data_helpers import prepare_examples, create_query
import spacy
from dotenv import load_dotenv

load_dotenv()

/Users/marcbraun/.conda/envs/732A81-Text_Mining/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Prepare Data

In [2]:
train1 = pd.read_csv('../../e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../../e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
dev = pd.read_csv('../../e-SNLI/dataset/esnli_dev.csv')
test = pd.read_csv('../../e-SNLI/dataset/esnli_test.csv')

train = train.dropna(subset=['Sentence1', 'Sentence2', 'Explanation_1'])
train = train.reset_index(drop=True)

np.random.seed(12345) # seed for numpy package
test_indices = list(np.random.choice(test.index.values, size=1000, replace=False))
test = test.loc[test_indices]
test = test.reset_index(drop=True)

# Analyse Predictions

In [6]:
nlp = spacy.load('en_core_web_sm')

In [78]:
docs = [nlp(test.iloc[i].Explanation_1) for i in range(20)]

In [79]:
for doc in docs:
    for chunk in doc.noun_chunks:
        print(chunk.text, '//', chunk.root.text, '//', chunk.root.dep_, '//', chunk.root.head.text)
    print('-----------------------------------------')

A bicycle // bicycle // nsubj // are
a bike // bike // conj // bicycle
the same object // object // attr // are
-----------------------------------------
a woman // woman // nsubj // has
a smile // smile // dobj // has
her face // face // pobj // on
she // she // nsubj // crying
-----------------------------------------
Jumping // Jumping // nsubj // is
an action // action // attr // is
a ball // ball // dobj // catch
-----------------------------------------
one person // person // attr // be
one man // man // nsubj // shaving
another man's hair // hair // dobj // shaving
-----------------------------------------
they // they // nsubj // analyze
the art // art // dobj // analyze
-----------------------------------------
A snowboarder // snowboarder // nsubj // is
a person // person // attr // is
a snowboarder // snowboarder // nsubj // sails
the air // air // pobj // through
they // they // nsubj // snowboarding
a person // person // nsubj // snowboarding
-----------------------------

In [64]:
for i in range(10):
    doc = nlp(test.iloc[i].Explanation_1)
    html = spacy.displacy.render(doc, style='dep')
    with open('data_vis' + str(i) + '.html', 'w') as f:
        f.write(html)

TypeError: write() argument must be str, not None